# Modelo de Equilibrio Radiativo

Baseado nas notas de [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook) by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.

____________
<a id='section1'></a>

## 1. Perfil observado da temperatura média anual global
____________

Perfil de temperatura obtido da reanalises do NCEP [Radiation notes](https://brian-rose.github.io/ClimateLaboratoryBook/courseware/radiation.html).

Gráfico Skew-T.

In [4]:
!pip install metpy # util em meteorologia (e.g. Skew-T)

In [5]:
!pip install netcdf4 # útil com manipulação de dados climáticos, atmosféricos, etc.

     ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 2.3/5.4 MB 12.1 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [6]:
!pip install climlab # putil com sub-modelos climáticos e meteorologia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.5 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-

In [7]:
#  This code is used just to create the skew-T plot of global, annual mean air temperature
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from metpy.plots import SkewT
ncep_url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/"
ncep_air = xr.open_dataset( ncep_url + "pressure/air.mon.1981-2010.ltm.nc", use_cftime=True)
#  Take global, annual average
coslat = np.cos(np.deg2rad(ncep_air.lat))
weight = coslat / coslat.mean(dim='lat')
Tglobal = (ncep_air.air * weight).mean(dim=('lat','lon','time'))

ImportError: ignored

In [ ]:
def make_skewT():
    fig = plt.figure(figsize=(9, 9))
    skew = SkewT(fig, rotation=30)
    skew.plot(Tglobal.level, Tglobal, color='black', linestyle='-', linewidth=2, label='Observations')
    skew.ax.set_ylim(1050, 10)
    skew.ax.set_xlim(-90, 45)
    # Add the relevant special lines
    skew.plot_dry_adiabats(linewidth=0.5)
    skew.plot_moist_adiabats(linewidth=0.5)
    #skew.plot_mixing_lines()
    skew.ax.legend()
    skew.ax.set_xlabel('Temperature (degC)', fontsize=14)
    skew.ax.set_ylabel('Pressure (hPa)', fontsize=14)
    return skew

In [ ]:
skew = make_skewT()

Vamos trabalhar com um **Modelo de Coluna Simples** para entender alguns conceitos como:

- Quais são os fatores que determinam este perfil de temperatura?
- Este perfil muda se alterarmos os gases da atmosfera?
- Qual o papel relativo da **radiação** e da **dinâmica** no estabelecimento deste perfil?

Vamos avaliar inicialmente apenas o papel da **radiação**. Vamos calcular a temperatura baseado no **equilíbrio radiativo**.

____________

## 2. Equilíbrio Radiativo -- conceitos
____________

Modelos de transferencia radiativa dividem a atmosfera em colunas de ar, e calculam a **emissão** and **absorão** da radiação em cada camada.


![Sketch of layer model](http://www.atmos.albany.edu/facstaff/brose/classes/ENV415_Spring2018/images/2layerAtm_sketch.png)

O conceito de **equilíbrio radiativo** significa que **ignoramos todas as formas de troca de calor exceto a radiação**, e perguntamos qual o perfil de temperatura deve ocorrer nestas condições?

Podemos responder a esta questao usando um modelo de transferencia radiativa para calcular os feixes de radiaçao de onda curta e radiaçao de onda longa, e avaliar o aquecimento ou resfriamento de cada camada associado com as fontes e sumidouros de energia radiativa.

Basicamente, encontramos o equilíbrio radiativo quando a **energia é recebida e perdida por radiação na mesma taxa em cada camada**.

Devido á complicada dependencia da emissao e absorção em funçao das bandas de comprimento de onda da radiaçao e os vários gases, o feixe é dividido em muitas bandas espectrais do espectro eletromagnético.

Não vamos entrar em todos os detalhes neste experimento devido á sua complexidade, mas vamos usar um **modelo** que representa estes processos no mesmo nível de detalhes que são usados em vários modelos.

____________

## 3. Modelo de coluna baseado no climlab
____________

### Radiation models in climlab

Vamos usar o **climlab** para executar um **modelo de radiação**, que leva em conta as propriedades de absorçao dos gases em funçao das bandas espectrais.

climlab permite usar dois modelos [radiation codes](https://climlab.readthedocs.io/en/latest/api/climlab.radiation.Radiation.html):

- O modelo [CAM3 radiation module](https://climlab.readthedocs.io/en/latest/api/climlab.radiation.CAM3.html) do NCAR (modelo de radiação usado no Modelo CESM)
- O modelo [RRTMG (Rapid Radiative Transfer Model)](https://climlab.readthedocs.io/en/latest/api/climlab.radiation.RRTMG.html) que é usado em muitos outros modelos como o WRF.

Links da documentação online [climlab documentation](http://climlab.readthedocs.io/en/latest/intro.html).

Vamos usar o modelo de radiação denominado [Rapid Radiative Transfer Model](http://rtweb.aer.com/rrtm_frame.html) ou RRTMG. Este modelo é usado em muitos modelos mais sofisticados que fazem previsão numérica de tempo e estudos climáticos.

climlab permite o uso por Python do código do modelo RRTMG.

### Vapor dágua

Precisamos incluir perfis de vapor dágua pois é um importante gás que interage com a radiação em várias bandas espectrais.

Usamos um perfil de umidade específica obtido do modelo CESM.

In [ ]:
#  Load the model output as we have done before
cesm_data_path = "http://thredds.atmos.albany.edu:8080/thredds/dodsC/CESMA/"
atm_control = xr.open_dataset(cesm_data_path + "cpl_1850_f19/concatenated/cpl_1850_f19.cam.h0.nc")
#  The specific humidity is stored in the variable called Q in this dataset:
print(atm_control.Q)

Adotamos um perfil média global da umidade específica:

In [ ]:
# Take global, annual average of the specific humidity
weight_factor = atm_control.gw / atm_control.gw.mean(dim='lat')
Qglobal = (atm_control.Q * weight_factor).mean(dim=('lat','lon','time'))
# Take a look at what we just calculated ... it should be one-dimensional (vertical levels)
print(Qglobal)

Figura:

In [ ]:
fig, ax = plt.subplots()
#  Multiply Qglobal by 1000 to put in units of grams water vapor per kg of air
ax.plot(Qglobal*1000., Qglobal.lev)
ax.invert_yaxis()
ax.set_ylabel('Pressure (hPa)')
ax.set_xlabel('Specific humidity (g/kg)')
ax.grid()

A Figura apresenta um perfil típico climatológico de umidade.

Baseado na figura, onde localiza-se a maior parte do vapor dágua?

### Modelo de coluna simples:

Here we will create the grid and state variables (air and surface temperature) for our single-column model.

In [ ]:
import climlab
#  Make a model on same vertical domain as the GCM
mystate = climlab.column_state(lev=Qglobal.lev, water_depth=2.5)
print(mystate)

In [ ]:
radmodel = climlab.radiation.RRTMG(name='Radiation (all gases)',  # give our model a name!
                              state=mystate,   # give our model an initial condition!
                              specific_humidity=Qglobal.values,  # tell the model how much water vapor there is
                              albedo = 0.25,  # this the SURFACE shortwave albedo
                              timestep = climlab.constants.seconds_per_day,  # set the timestep to one day (measured in seconds)
                             )
print(radmodel)

### Explore o modelo de coluna simples

Avalie as propriedades do modelo:

In [ ]:
#  Here's the state dictionary we already created:
radmodel.state

In [ ]:
#  Here are the pressure levels in hPa
radmodel.lev

A variável `absorber_vmr` contém a *razão de mistura por volume* de todos os gases que interagem com a radiação na coluna do modelo:

In [ ]:
radmodel.absorber_vmr

A maioria dos gases apresentam apenas uma concentração pois consideramos que são **bem misturados** na atmosfera.

Exceto o ozonio, que possui uma estrutura vertical. Veja

In [ ]:
#  E.g. the CO2 content (a well-mixed gas) in parts per million
radmodel.absorber_vmr['CO2'] * 1E6

### Python exercise: plot the ozone profile

Make a simple plot showing the vertical structure of ozone, similar to the specific humidity plot we just made above.

In [ ]:
# here is the data you need for the plot, as a plain numpy arrays:
print(radmodel.lev)
print(radmodel.absorber_vmr['O3'])

O vapor dágua também é um importante gás que possui uma estrutura vertical e armazenado como `specific_humidity`:

In [ ]:
#  specific humidity in kg/kg, on the same pressure axis
print(radmodel.specific_humidity)

### O modelo de radiação RRTMG possui vários outros parâmetros de entrada

For details you can look at the [documentation](http://climlab.readthedocs.io/en/latest/api/climlab.radiation.radiation.html)

In [ ]:
for item in radmodel.input:
    print(item)

Muitos parâmetros controlam o efeito **radiativo das nuvens**.

Inicialmente não vamos considerar **o efeito das nuvens**:

In [ ]:
#  This is the fractional area covered by clouds in our column:
radmodel.cldfrac

____________

1.   Item da lista
2.   Item da lista



## 4. Equilíbrio radiativo na coluna
____________

### Avançar o modelo no tempo!

Here are the current temperatures (initial condition):

In [ ]:
radmodel.Ts

In [ ]:
radmodel.Tatm

Now let's take a single timestep:

In [ ]:
radmodel.step_forward()

In [ ]:
radmodel.Ts

A superfície ficou mais quente!

Vamos olhar os resultados deste avanço no tempo:

### Variáveis diagnósticas em nosso modelo de coluna

O climlab permite muitas variáveis `diagnostics` . Vejamos algumas `xarray`:

In [ ]:
climlab.to_xarray(radmodel.diagnostics)

O principal "objetivo" do modelo de transferência radiativa é calcular os fluxos de radiação de onda curta e radiação de onda longa entre cada camada do modelo.

Por exemplo:

In [ ]:
climlab.to_xarray(radmodel.LW_flux_up)

Estes são os fluxos de onda longa para cima em W/m2.

Porque temos 27 pontos, quando temos apenas 26 níveis de pressão?

In [ ]:
radmodel.lev

In [ ]:
radmodel.lev_bounds

O último termo de fluxo (do vetor) representa o **fluxo para cima a partir da superfície para o primeiro nível**:

In [ ]:
radmodel.LW_flux_up[-1]

O valor é aproximadamente 390 W m$^{-2}$.

Porque?

In [ ]:
sigma = 5.67E-8
sigma * 288**4

A temperatura da superfície foi inicializada com o valor de 288 K, e é considerada emitir como um corpo negro no modelo.

Qual é o valor do fluxo a partir do topo da atmosfera para o espaço?

Duas maneiras de acessar a informação:

In [ ]:
radmodel.LW_flux_up[0]

In [ ]:
radmodel.OLR

Existe várias outras variáveis diagnósticas também para os fluxos de radiação de onda curta.

Um diagnóstico de grande interesse é **o balanço de energia no topo da atmosfera**:

In [ ]:
radmodel.ASR - radmodel.OLR

***O modelo está ganhando ou perdendo energia?***

### Integre até o equilíbrio

Devemos avançar no tempo até que o modelo esteja muito próximo do balanço de energia.

Podemos usar um loop até `while` e avaliar este balanço no topo da atmosfera:

In [ ]:
while np.abs(radmodel.ASR - radmodel.OLR) > 0.01:
    radmodel.step_forward()

Check the energy budget again:

In [ ]:
#  Check the energy budget again
radmodel.ASR - radmodel.OLR

Indeed, the imbalance is now small.

### Compare a temperatura de equilíbrio radiativo com as observações

Here's a helper function we'll use to add model temperature profiles to our skew-T plot:

In [ ]:
def add_profile(skew, model, linestyle='-', color=None):
    line = skew.plot(model.lev, model.Tatm - climlab.constants.tempCtoK,
             label=model.name, linewidth=2)[0]
    skew.plot(1000, model.Ts - climlab.constants.tempCtoK, 'o',
              markersize=8, color=line.get_color())
    skew.ax.legend()

In [ ]:
skew = make_skewT()
add_profile(skew, radmodel)
skew.ax.set_title('Pure radiative equilibrium', fontsize=18);

What do you think about this model -- data comparison?

____________

## 5. Efeitos de diferentes gases no perfil de equilíbrio radiativo
____________

**Modelos geralmente são usados para experimentar e testar!**

Acabamos de construir um modelo para radiação que interage com vários gases. *Podemos aprender melhor seus efeitos simplesmente retirando cada um deles.*

### Equilíbrio radiativo sem o vapor dágua

In [ ]:
# Make an exact clone of our existing model
radmodel_noH2O = climlab.process_like(radmodel)
radmodel_noH2O.name = 'Radiation (no H2O)'
print(radmodel_noH2O)

In [ ]:
#  Here is the water vapor profile we started with
radmodel_noH2O.specific_humidity

Now get rid of the water entirely!

In [ ]:
radmodel_noH2O.specific_humidity *= 0.

In [ ]:
radmodel_noH2O.specific_humidity

Executamos novamente este novo modelo até o equilíbrio:

In [ ]:
#  it's useful to take a single step first before starting the while loop
#   because the diagnostics won't get updated
#  (and thus show the effects of removing water vapor)
#  until we take a step forward
radmodel_noH2O.step_forward()
while np.abs(radmodel_noH2O.ASR - radmodel_noH2O.OLR) > 0.01:
    radmodel_noH2O.step_forward()

In [ ]:
radmodel_noH2O.ASR - radmodel_noH2O.OLR

In [ ]:
skew = make_skewT()
for model in [radmodel, radmodel_noH2O]:
    add_profile(skew, model)

What do you think you can learn from this about the radiative role of water vapor?

### Exerccício: Modelo radiativo sem o ozonio

Siga os passos acima e remova o ozonio!

Faça outro diagrama skew-T comparando os resultados dos três modelos.

Fica como sugestão remover o ozonio e o vapor dágua juntos!

____________

## 6. Resumo sobre os resultados do equilíbrio radiativo
____________

- Usamos o modelo `RRTMG` que contém perfis prescritos de gases absorvedores para calcular perfis de temperatura baseano no ** equilíbrio de radiação** apenas.
- Equilíbrio Radiativo significa a temperatura que a superfície e as colunas de ar devem ter **se a radiação for o único processo físico que transfere energia entre as camadas**.
- Estimamos vários casos de equilibrio radiativo, com e sem alguns gases
    - O perfil sem o vapor dágua é **muito frio na superfície e baixa troposfera**, mas aproximadamente a mesma na estratosfera
    - O perfil sem o ozonio é **muito frio na estratosfera**, mas aproximadamente o mesmo próximo da superfície.
    - De fato não existe estratosfera sem a presença do ozonio! A temperatura é isotérmica nos níveis mais altos do perfil de temperatura.

No entanto, podemos concluir que **nenhum destes perfis de equilíbrio radiativo se parecem com o perfil observado**.

Isto sugere que outros processos físicos (além da radiação) são importantes na determinação do perfil de temperatura.

Os diagramas skew-T mostram claro que **todos os perfis são estaticamente instáveis próximo da superfície**.

O próximo passo é portanto olhar nos efeitos da mistura convectiva nos perfis de temperatura.

____________

## Credits

This notebook is part of [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook), an open-source textbook developed and maintained by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.

It is licensed for free and open consumption under the
[Creative Commons Attribution 4.0 International (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/) license.

Development of these notes and the [climlab software](https://github.com/brian-rose/climlab) is partially supported by the National Science Foundation under award AGS-1455071 to Brian Rose. Any opinions, findings, conclusions or recommendations expressed here are mine and do not necessarily reflect the views of the National Science Foundation.
____________